In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
trn_data = pd.read_csv('./data/train.csv')
tst_data= pd.read_csv('./data/test.csv')
sub= pd.read_csv('./data/sample_submission.csv')
snp = pd.read_csv('./data/snp_info.csv')

['SNP_01',
 'SNP_02',
 'SNP_03',
 'SNP_04',
 'SNP_05',
 'SNP_06',
 'SNP_07',
 'SNP_08',
 'SNP_09',
 'SNP_10',
 'SNP_11',
 'SNP_12',
 'SNP_13',
 'SNP_14',
 'SNP_15']

In [6]:
# trn_data.info()
# tst_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      262 non-null    object
 1   father  262 non-null    int64 
 2   mother  262 non-null    int64 
 3   gender  262 non-null    int64 
 4   trait   262 non-null    int64 
 5   SNP_01  262 non-null    object
 6   SNP_02  262 non-null    object
 7   SNP_03  262 non-null    object
 8   SNP_04  262 non-null    object
 9   SNP_05  262 non-null    object
 10  SNP_06  262 non-null    object
 11  SNP_07  262 non-null    object
 12  SNP_08  262 non-null    object
 13  SNP_09  262 non-null    object
 14  SNP_10  262 non-null    object
 15  SNP_11  262 non-null    object
 16  SNP_12  262 non-null    object
 17  SNP_13  262 non-null    object
 18  SNP_14  262 non-null    object
 19  SNP_15  262 non-null    object
 20  class   262 non-null    object
dtypes: int64(4), object(17)
memory usage: 43.1+ KB


In [8]:
trn_data.columns

Index(['id', 'father', 'mother', 'gender', 'trait', 'SNP_01', 'SNP_02',
       'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
       'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'class'],
      dtype='object')

In [14]:
trn_data.groupby('trait').sum()

C:\Users\deepBlue\AppData\Local\Temp\ipykernel_14856\1869791530.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  trn_data.groupby('trait').sum()


,father,mother,gender
trait,,,
1,0,0,0
2,0,0,0


In [15]:
seq = ['id', 'father', 'mother', 'gender', 'trait', 'SNP_01', 'SNP_02',
       'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
       'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'class']

# for i in range(len(seq)):
#     print(seq[i])
#     train.groupby(seq[i]).sum()

In [8]:
# 컬럼 나누기
one_hot_features = ['trait']

categorical_features = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [17]:
# 라벨 인코딩
def encode_categorical(train_df, test_df, categ_feat = categorical_features):

    encoder_dict = {}
    
    concat_data = pd.concat([trn_data[categ_feat], tst_data[categ_feat]])
    
    for col in concat_data.columns:
        print('Encoding: ', col, '...')
        encoder = LabelEncoder()
        encoder.fit(concat_data[col])
        encoder_dict[col] = encoder

        train_df[col + '_Enc'] = encoder.transform(train_df[col])
        test_df[col + '_Enc'] = encoder.transform(test_df[col])
    
    train_df = train_df.drop(columns = categ_feat, axis = 1)
    test_df = test_df.drop(columns = categ_feat, axis = 1)

    return train_df, test_df

In [18]:
trn_data, tst_data = encode_categorical(trn_data, tst_data, categorical_features)

Encoding:  SNP_01 ...
Encoding:  SNP_02 ...
Encoding:  SNP_03 ...
Encoding:  SNP_04 ...
Encoding:  SNP_05 ...
Encoding:  SNP_06 ...
Encoding:  SNP_07 ...
Encoding:  SNP_08 ...
Encoding:  SNP_09 ...
Encoding:  SNP_10 ...
Encoding:  SNP_11 ...
Encoding:  SNP_12 ...
Encoding:  SNP_13 ...
Encoding:  SNP_14 ...
Encoding:  SNP_15 ...


In [19]:
# 원핫
def one_hot(df, one_hot_categ):
    for col in one_hot_categ:
        tmp = pd.get_dummies(df[col], prefix = col)
        df = pd.concat([df, tmp], axis = 1)
    df = df.drop(columns = one_hot_categ)
    return df

In [21]:
trn_data = one_hot(trn_data, one_hot_features)
tst_data = one_hot(tst_data, one_hot_features)

In [25]:
trn_data = trn_data.drop(['father','mother','gender'], axis=1)
tst_data = tst_data.drop(['father','mother','gender'], axis=1)

In [27]:
## 파일 생성
import os
from datetime import datetime

def file_make(filename):
    now = datetime.now()
    date =  now.strftime('%Y-%m-%d')

    file_path = './'+ filename +'/' + date
    os.makedirs(file_path,exist_ok=True)
    print(file_path)
    return file_path

In [28]:
trn_path = file_make('data') + '/trn_data.csv'
tst_path = file_make('data') + '/tst_data.csv'

trn_data.to_csv(trn_path,index=False)
tst_data.to_csv(tst_path,index=False)

./data/2022-12-13
./data/2022-12-13
